In [1]:
!pip3 install transformers
!pip3 install librosa
from transformers import Wav2Vec2Tokenizer, Wav2Vec2ForCTC
import librosa as lb
import torch

     |████████████████████████████████| 4.0 MB 27.0 MB/s 
     |████████████████████████████████| 895 kB 54.0 MB/s 
     |████████████████████████████████| 6.5 MB 47.5 MB/s 
     |████████████████████████████████| 596 kB 48.8 MB/s 
     |████████████████████████████████| 77 kB 6.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import numpy as np

In [6]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [7]:
params = {
    'debug': False,
    'checkpoint': 'facebook/wav2vec2-base-960h',
    'max_len': 256,
    'batch_size': 32,
    'device': device
}

In [70]:
waveforms = [r'/content/Order! Speaker John Bercow sorgt im House of Commons für Ordnung (320 kbps).wav', r'/content/Order! Speaker John Bercow sorgt im House of Commons für Ordnung (320 kbps).wav']

In [79]:
class S2tDataset:
    def __init__(self, waveforms, max_len=params['max_len'], checkpoint=params['checkpoint']):
        self.waveforms = waveforms
        self.max_len = max_len
        self.checkpoint = checkpoint
        # Initialize the tokenizer
        self.tokenizer = Wav2Vec2Tokenizer.from_pretrained(self.checkpoint)
        self.num_examples = len(self.waveforms)

    def __len__(self):
        return self.num_examples

    def __getitem__(self, idx):
        waveform_path = self.waveforms[idx]
        print('HERE', waveform_path)
        
        # Read the sound file
        waveform, rate = lb.load(waveform_path, sr = 16000)

        # Tokenize the waveform
        input_values = self.tokenizer(waveform, return_tensors='pt').input_values

        return {'ids':input_values
                }

In [80]:
class S2tModel:
    def __init__(self, checkpoint=params['checkpoint']):
        self.checkpoint = checkpoint
         # Initialize the model
        self.model = Wav2Vec2ForCTC.from_pretrained(self.checkpoint)

In [71]:
model = S2tModel().model
# model.load_state_dict(torch.load(model_name))
model = model.to(params['device'])

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [93]:
model.eval()

test_dataset = S2tDataset(
    waveforms = waveforms
)

test_loader = DataLoader(
    test_dataset, 
    shuffle=False,
    pin_memory=True
)

temp_preds = None
l = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc=f'Predicting. '):
        ids= batch['ids'].squeeze(0).to(device)

        predictions = model(ids)

        # print(predictions, predictions.shape, type(predictions))
        logits = predictions.logits.detach().cpu().numpy()

        l.extend([list(p) for p in np.argmax(logits, axis=2)])
        

# print(*l, sep='\n')


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:748: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  FutureWarning,
Predicting. :   0%|          | 0/2 [00:00<?, ?it/s]

HERE /content/Order! Speaker John Bercow sorgt im House of Commons für Ordnung (320 kbps).wav


Predicting. :  50%|█████     | 1/2 [00:03<00:03,  3.66s/it]

HERE /content/Order! Speaker John Bercow sorgt im House of Commons für Ordnung (320 kbps).wav


Predicting. : 100%|██████████| 2/2 [00:06<00:00,  3.39s/it]


#**Inference:**

In [95]:
# Initialize the tokenizer
tokenizer = Wav2Vec2Tokenizer.from_pretrained('facebook/wav2vec2-base-960h')

# Take argmax value and decode into transcription
transcription = tokenizer.batch_decode(l)

# Print the output
print('\nOutput:',*transcription, sep='\n')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.



Output:
AND WILL BE HEARD ORDER ODER THE ORABLE GENTLEMAN HAS GOT TO LEARN THE ART OF PATIENCE AND IF HE IS PATIENT
AND WILL BE HEARD ORDER ODER THE ORABLE GENTLEMAN HAS GOT TO LEARN THE ART OF PATIENCE AND IF HE IS PATIENT


/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:748: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  FutureWarning,
